# Welcome — Using the Memories.AI API

This Jupyter notebook guides you through using the [Memories.AI API](https:memories.ai/docs). It contains upload, search, videochat, transcibe, and several utility tools, for you to run and adapt.

## Prerequisites

- Python 3.8+ running in a Jupyter environment (Notebook, JupyterLab, or Colab).
- Package:
  - `requests`

- An API key: Save it here.


In [ ]:
API_KEY = "" # Your memories.ai API key

This is an example video (and its URL) that we will be working on. You can click run and see the video.


In [ ]:
from IPython.display import HTML
# Note that you don't need to modify this block.
VIDEO_URL = "https://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ElephantsDream.mp4"

html = f"""
<div style="max-width:720px;">
  <video controls playsinline style="width:100%; height:auto;" preload="metadata" poster="">
    <source src="{VIDEO_URL}" type="video/mp4">
    Your browser does not support HTML5 video. <a href="{VIDEO_URL}">Download</a>
  </video>
</div>
"""
display(HTML(html))

## Upload

Let’s start by uploading the video from the URL shown above.

If the upload is successful, the returned JSON will include:

```json
"videoNo": "...",
"videoName": "...",
"success": true


In [ ]:
import requests  
headers = {"Authorization": API_KEY} 

# Video file details  
data = {  
    "url": "http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ElephantsDream.mp4"
    # "callback": "<YOUR_CALLBACK_URI>" # Optional
}  

response = requests.post(  
    "https://api.memories.ai/serve/api/video/upload_url",  
    data=data,  
    headers=headers  
)  

print(response.json())  

{'code': '0000', 'msg': 'success', 'data': {'videoNo': 'VI611471886560694272', 'videoName': 'ElephantsDream'}, 'success': True, 'failed': False}


## Check video parsing status

Before proceeding, confirm that the video has been fully parsed by our model.  

Run the next code cell to check the status.  
Continue only when you see:

```json
"status": "PARSE"

In [ ]:
headers = {"Authorization": API_KEY} 
params = {
    "video_no": "VI611471886560694272"  # Replace with the actual video number from the upload response,
}
response = requests.post("https://api.memories.ai/serve/api/video/list_videos", headers=headers, json=params)
print(response.json())

{'code': '0000', 'msg': 'success', 'data': {'videos': [{'duration': '653', 'size': '169612362', 'status': 'PARSE', 'cause': 'null', 'video_no': 'VI611471886560694272', 'video_name': 'ElephantsDream', 'create_time': '1755245460412'}], 'current_page': 1, 'page_size': 200, 'total_count': '1'}, 'success': True, 'failed': False}


## Search

You can perform a **semantic search** across all uploaded videos to find the most relevant matches.

In this example, we’ll search for the video we just uploaded by providing a short text description of its content.


In [ ]:
headers = {"Authorization": API_KEY} 
json_body = {
    "search_param": "Proog (an older guide) and Emo (a younger protege) explore a surreal, vast machine",  # The search query
    "search_type": "BY_VIDEO" # 'BY_AUDIO' or 'BY_VIDEO' or 'BY_CLIP'
}

response = requests.post(
    "https://api.memories.ai/serve/api/video/search",
    headers=headers,
    json=json_body
)

print(response.json())

{'code': '0000', 'msg': 'success', 'data': [{'videoNo': 'VI611471886560694272', 'videoName': 'ElephantsDream', 'startTime': 'null', 'endTime': 'null'}, {'videoNo': 'VI611145110490705920', 'videoName': 'ElephantsDream', 'startTime': 'null', 'endTime': 'null'}, {'videoNo': 'VI605983006653673472', 'videoName': '足球', 'startTime': 'null', 'endTime': 'null'}], 'failed': False, 'success': True}


## VideoChat Prompt Example

You can interact with the **Memories.AI Chat API** to ask questions about a specific video.

In the example below, we’ll send a promptregarding highlights to the API and receive a response generated from the video’s parsed content.


In [16]:
import json
headers = {
    "Authorization": API_KEY,
    "Content-Type": "application/json",
}

payload = {
    "video_nos": ["VI611471886560694272"],  # List of video IDs to chat about
    "prompt": "Find the highlights of this video and give timestamps"  # User query
}

response = requests.post(
    "https://api.memories.ai/serve/api/v1/chat",
    headers=headers,
    data=json.dumps(payload),
    stream=False
)

if response.status_code != 200:
    print(response.status_code)
    print(response.text)
else:
    try:
        for line in response.iter_lines(decode_unicode=True):
            if line:
                print(line)
                if line.strip().lower() == 'data:"done"':
                    print("\n")
                    break
                if line.startswith("data:"):
                    print(line.replace("data:", "").strip(), end="", flush=True)
    except Exception as e:
        print(str(e))

{"code":"0000","msg":"success","data":{"role":"ASSISTANT","content":"Here are the key highlights with timestamps (approximate) and why each moment matters — prioritized by narrative importance.\n\nHighlights & timestamps\n- 00:00–00:30 — Opening atmosphere\n  - What happens: Opens on a surreal, metallic world of wires, lights and suspended platforms that establishes tone and scale.\n  - Why it matters: Sets the film’s mood and visual language — ominous, mechanical, labyrinthine.\n\n- 00:30–02:00 — Introduction of the two characters\n  - What happens: Proog (older, bald man in a dark green coat) and Emo (younger, gaunt man in a light tunic) appear and begin interacting.\n  - Why it matters: Establishes the mentor/guide dynamic and their contrasting attitudes toward the machine.\n\n- 02:00–04:30 — Exploration of the machinery and environment\n  - What happens: The pair move across pathways and platforms, encounter metallic bird-like figures, spinning cylinders, and tentacle-like vines.\n

## Video Marketer

You can explore our **Large Visual Memory Model** and **Agentic QA System** without uploading any videos.

With this API, you can chat directly with insights drawn from the entire 1-million-video dataset.


In [ ]:
headers = {
    "Authorization": API_KEY,
    "Content-Type": "application/json",
}

payload = {
    "prompt": "where is the war?",  # User query
}

response = requests.post(
    "https://api.memories.ai/serve/api/v1/marketer_chat",
    headers=headers,
    data=json.dumps(payload),
    stream=False
)

print(response.status_code)
print(response.json())

200
{'code': '0000', 'msg': 'success', 'data': {'role': 'ASSISTANT', 'content': 'Do you mean a specific war? Here are the likely meanings and where each is located:\n\n- If you mean the discussion clip titled "The Price of War" — @thedailyshow — it’s not about a single battlefield; it’s a conversation about U.S. service members and the nationwide costs and sacrifices of military service, not a specific geographic war. Quote: "@thedailyshow — \'every individual who enlists... makes the same sacrifice.\'"\n\n- If you mean "World War I" mentioned by @bbc — that war took place mainly in Europe (notably the Western Front in France and Belgium and the Eastern Front in what was then the Russian Empire), with additional campaigns in the Middle East, Africa, Asia and at sea. Quote: "@bbc — \'The Great War\' — a conflict of unprecedented scale."\n\n- If you mean contemporary, ongoing conflicts (major ones as of mid‑2024):  \n  - Russia’s full‑scale war in Ukraine — fighting in Ukraine.  \n  - Is

# Video transcription
You can use the API to transcribe the video's visual content into text.

In [13]:
headers = {"Authorization": API_KEY}
params = {"video_no": "VI611471886560694272"}

response = requests.get("https://api.memories.ai/serve/api/video/get_video_transcription", headers=headers, params=params)

print("Status:", response.status_code)
try:
    print("Video Transcription Response:", response.json())
except Exception:
    print("Response Text:", response.text)

Status: 200
Video Transcription Response: {'code': '0000', 'msg': 'success', 'data': {'videoNo': 'VI611471886560694272', 'transcriptions': [{'index': 0, 'content': 'The screen displays a textured wall with the words "the orange open movie project presents" in a stylized font. The background is dark and appears to be made of cracked tiles, with some metallic or mechanical elements visible on the sides.', 'startTime': '0', 'endTime': '26'}, {'index': 1, 'content': 'A close-up shot focuses on the face of an elderly, bald man with a weathered appearance. He is looking downwards with a concerned expression, and his features are blurred due to motion.', 'startTime': '26', 'endTime': '28'}, {'index': 2, 'content': "The scene shifts to a close-up of a man's face, revealing blue eyes and a prominent nose. He appears to be looking directly at the viewer with a surprised or alarmed expression. The background is dark and out of focus.", 'startTime': '28', 'endTime': '29'}, {'index': 3, 'content': 

## Audio Transcription

Use the API to convert a video’s **spoken audio** into text.

> **Note:** The video must contain audible speech for transcription to work correctly.


In [ ]:
headers = {"Authorization": API_KEY}
params = {"video_no": "VI611471886560694272"}

response = requests.get("https://api.memories.ai/serve/api/v1/get_audio_transcription", headers=headers, params=params)

print("Status:", response.status_code)
try:
    print("Video Transcription Response:", response.json())
except Exception:
    print("Response Text:", response.text)

Status: 200
Video Transcription Response: {'code': '0000', 'msg': 'success', 'data': None, 'success': True, 'failed': False}


## Generate Video Summary

Use the API to create a concise summary of a video’s content.

The `type` parameter accepts:
- `"TOPIC"` 
- `"CHAPTER"` 


In [ ]:
headers = {"Authorization": API_KEY}
params = {
    "video_no": "VI611471886560694272",
    "type": "TOPIC"
}

response = requests.get("https://api.memories.ai/serve/api/video/generate_summary", headers=headers, params=params)

print("Status:", response.status_code)
try:
    print("Summary Response:", response.json())
except Exception:
    print("Response Text:", response.text)

Status: 200
Summary Response: {'code': '0000', 'msg': 'success', 'data': {'summary': 'This animated short film follows the journey of two characters, Emo and Proog, through a surreal and industrial world filled with intricate machinery and strange organic elements. Proog attempts to guide Emo, who struggles to perceive the beauty and purpose of their surroundings. The film explores themes of perception, trust, and the struggle to find meaning in a complex and unsettling environment. The narrative is accompanied by abstract visuals and cryptic dialogue, culminating in a sense of uncertainty and the unknown.', 'items': [{'description': "• Proog guides Emo through a complex, metallic environment.\n• Proog urges Emo to get up, stating it's not safe.", 'title': 'The Journey Begins'}, {'description': "• Proog emphasizes the importance of listening to the sounds of the machine and one's own breathing.\n• The dialogue touches on the potential danger they face.", 'title': 'The Importance of Lis

# Utility
There are also utility APIs for managing and retrieving metadata about videos and sessions in the Memories.ai platform.

# List Videos
You can retrieve a paginated list of videos that have been uploaded to the platform. You can optionally filter the results by video name, video ID, folder, or processing status.

In [ ]:
headers = {"Authorization": API_KEY}

json_body = {
    "page": 1,
    "size": 200,
    # "video_name": "<VIDEO_NAME>",
    # "video_no": "<VIDEO_ID>",
    # "unique": "<UNIQUE_ID>",
    # "status": "<STATUS>",
}

response = requests.post("https://api.memories.ai/serve/api/video/list_videos", headers=headers, json=json_body)
print(response.json())

{'code': '0000', 'msg': 'success', 'data': {'videos': [{'duration': '653', 'size': '169612362', 'status': 'PARSE', 'cause': 'null', 'video_no': 'VI611471886560694272', 'video_name': 'ElephantsDream', 'create_time': '1755245460412'}, {'duration': '426', 'size': '208935734', 'status': 'PARSE', 'cause': 'null', 'video_no': 'VI611165958140055552', 'video_name': 'onlychats', 'create_time': '1755172519391'}, {'duration': '426', 'size': '208935734', 'status': 'PARSE', 'cause': 'null', 'video_no': 'VI611154695637032960', 'video_name': 'onlychats', 'create_time': '1755169834208'}, {'duration': '0', 'size': '10768051', 'status': 'FAIL', 'cause': 'Indexing failed. ', 'video_no': 'VI611151298850443264', 'video_name': 'onlychats', 'create_time': '1755169024347'}, {'duration': '653', 'size': '169612362', 'status': 'PARSE', 'cause': 'null', 'video_no': 'VI611145110490705920', 'video_name': 'ElephantsDream', 'create_time': '1755167548927'}, {'duration': '0', 'size': '169612362', 'status': 'FAIL', 'cau

# List Chat Sessions
You can retrieve a paginated list of historical chat sessions, including metadata such as session IDs, timestamps, and related videos.

In [ ]:
headers = {"Authorization": API_KEY}

params = {
    "page": "1"
}

response = requests.get("https://api.memories.ai/serve/api/video/list_sessions", headers=headers, params=params)
print(response.json())

{'code': '0000', 'msg': 'success', 'data': {'sessions': [{'sessionId': '608224587769520128', 'title': 'Summarize this video'}, {'sessionId': '564586841416663040', 'title': 'what is this video about'}, {'sessionId': '553235038557175808', 'title': 'Advertisement Analysis and Recommendations'}, {'sessionId': '553235038557175809', 'title': 'Analysis of Zuckerberg Interview'}, {'sessionId': '553235038557175810', 'title': 'Discussion on MV Story and Concept'}, {'sessionId': '553235038557175811', 'title': 'Learn How to Make Sushi'}, {'sessionId': '553235038557175812', 'title': 'Neural Network Course Study'}, {'sessionId': '553235038557175813', 'title': "Tesla's EV and robot advancements"}, {'sessionId': '553235038557175814', 'title': 'Threads of Time: A Journey Back'}, {'sessionId': '553235038557175815', 'title': 'Analysis of Interstellar'}, {'sessionId': '553235038557175816', 'title': 'Summary of Altman'}], 'current_page': 1, 'page_size': 20, 'total_count': '11'}, 'success': True, 'failed': 

# Get Session Detail
You can retrieve detailed metadata about a specific video session.
From last block, we can choose a sessionId

In [ ]:
headers = {"Authorization": API_KEY}
params = {"sessionId": 553235038557175811}

response = requests.get("https://api.memories.ai/serve/api/video/get_session_detail", headers=headers, params=params)

if response.status_code == 200:
    print("Session detail retrieved successfully:")
    print(response.json())
else:
    print("Failed to retrieve session detail:", response.status_code)
    print(response.text)

Session detail retrieved successfully:
{'code': '0000', 'msg': 'success', 'data': {'title': 'Learn How to Make Sushi', 'messages': [{'role': 'user', 'content': 'What is this video about'}, {'role': 'assistant', 'content': 'This video is a sushi-making tutorial. A professional chef demonstrates how to make kappamaki (cucumber rolls) and tekkamaki (tuna rolls) at home, covering techniques from preparing the sushi rice to rolling and cutting the sushi.  He emphasizes the importance of properly cooked rice and using sushi-grade fish. He also provides tips for handling sushi rice and achieving a good roll.\n'}, {'role': 'user', 'content': 'Which step in the sushi making process is more difficult for a novice?'}, {'role': 'assistant', 'content': "Handling the sushi rice is one of the most difficult parts of making sushi for a novice.  It takes practice to get the right amount of moisture on your hands so the rice doesn't stick.\n"}], 'session_id': '553235038557175811'}, 'failed': False, 'suc

# Delete Video
You can delete all raw and derived data associated with the specified videoNos in the request. Once the API call is successfully completed, no data related to the deleted videos will be retained.

In [ ]:
headers = {"Authorization": API_KEY}  
# List of video IDs to delete
data = ["VI611165958140055552","VI611154695637032960"]

response = requests.post(
    "https://api.memories.ai/serve/api/video/delete_videos",
    headers=headers,
    json=data
)

print(response.json())

{'code': '0000', 'msg': 'success', 'data': None, 'success': True, 'failed': False}
